In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
store = pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv')
past_sales_data = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv')
future_sales_data = pd.read_csv('/kaggle/input/rossmann-store-sales/test.csv')
past_sales_data['Date'] = pd.to_datetime(past_sales_data['Date'], format='%Y-%m-%d')
future_sales_data['Date'] = pd.to_datetime(future_sales_data['Date'], format='%Y-%m-%d')
store.shape, past_sales_data.shape

In [ ]:
#past_sales_data.head(3)

In [ ]:
#future_sales_data['Date'].describe()

In [ ]:
#store.head()

In [ ]:
#store.describe()

In [ ]:
#store.isna().sum()

In [ ]:
#store.isna().sum() / store.shape[0] * 100

In [ ]:
#store[store['Promo2SinceWeek'].isna()].head()

In [ ]:
#store[~store['Promo2SinceWeek'].isna()].head()

In [ ]:
#past_sales_data.head()

In [ ]:
#past_sales_data['Store'].nunique()

In [ ]:
#past_sales_data.isna().sum()

In [ ]:
#single_store_sales = past_sales_data[past_sales_data['Store'] == 1]
#single_store_sales.sort_values('Date').set_index('Date')['Sales'].plot.line(figsize=(15,5))

In [ ]:
#single_store_sales[single_store_sales['Sales'] == 0]

In [ ]:
#single_store_sales.groupby(['DayOfWeek']).agg(avg_sales=('Sales', 'mean'))

In [ ]:
#past_sales_data.groupby('Store').agg(total_sales = ('Sales', 'mean')).sort_values(['total_sales'], ascending=False)

In [ ]:
#past_sales_data.describe(include=object)

In [ ]:
#past_sales_data['StateHoliday'].unique()

In [ ]:
#past_sales_data[past_sales_data['StateHoliday']==0]

In [ ]:
#past_sales_data[past_sales_data['Open']==0]['Sales']

In [ ]:
#past_sales_data['DayOfWeek_Text'] = past_sales_data['Date'].dt.strftime('%A')
#past_sales_data.head()

In [ ]:
#past_sales_data[past_sales_data['DayOfWeek'] == 7]

In [ ]:
#past_sales_data[past_sales_data['DayOfWeek'] ==7]['Open'].value_counts()

In [ ]:
#store.head()

In [ ]:
#past_sales_data[past_sales_data['DayOfWeek'] ==7][['Store', 'Open']].drop_duplicates()['Open'].value_counts()

In [ ]:
#past_sales_data[(past_sales_data['DayOfWeek']==7) & (past_sales_data['Open']==1)]['Store'].unique()

In [ ]:
#past_sales_data['Date'].describe()

### Data Preprocessing

In [ ]:
#store.head()

In [ ]:
#store.isna().sum()

In [ ]:
#store_new['CompetitionOpenSinceYear'].mode().iloc[0]
#store_new['CompetitionOpenSinceYear'].mode().tolist()[0]

In [ ]:
store_new = store.copy()
store_new['CompetitionDistance'] = store_new['CompetitionDistance'].fillna(0)
store_new['CompetitionOpenSinceMonth'] = store_new['CompetitionOpenSinceMonth'].fillna(0)
competition_year_mode = store_new['CompetitionOpenSinceYear'].mode().iloc[0]
store_new['CompetitionOpenSinceYear'] = store_new['CompetitionOpenSinceYear'].fillna(competition_year_mode)


store_new['Promo2SinceWeek'] = store_new['Promo2SinceWeek'].fillna(0)
promo2year_mode = store_new['Promo2SinceYear'].mode().iloc[0]
store_new['Promo2SinceYear'] = store_new['Promo2SinceYear'].fillna(promo2year_mode)
promo_interval_mode = store_new['PromoInterval'].mode().iloc[0]
store_new['PromoInterval'] = store_new['PromoInterval'].fillna(promo_interval_mode)
store_new.isna().sum()

In [ ]:
#past_sales_data.head()

In [ ]:
#store_new.head()

In [ ]:
past_sales_data_comb = pd.merge(left=past_sales_data, right=store_new,
                               how='left', on=['Store'])
past_sales_data_comb.head()

In [ ]:
#past_sales_data.shape, past_sales_data_comb.shape

In [ ]:
#past_sales_data['StateHoliday'].unique()

In [ ]:
#past_sales_data_comb['StateHoliday'].unique()

In [ ]:

past_sales_data_comb = pd.merge(left=past_sales_data, right=store_new,
                               how='left', on=['Store'])
past_sales_data_comb['StateHoliday'] = past_sales_data_comb['StateHoliday'].replace({0: '0'})
past_sales_data_comb['sales_month'] = past_sales_data_comb['Date'].dt.month
cols_ignore = ['Date', 'Customers', 'Store']
past_sales_data_comb = past_sales_data_comb.drop(cols_ignore, axis=1)
past_sales_data_dummies = pd.get_dummies(past_sales_data_comb, drop_first=True)
past_sales_data_dummies.shape, past_sales_data_comb.shape

In [ ]:
#past_sales_data_dummies.head()

In [ ]:
#past_sales_data_comb['StateHoliday'].unique()

### Model Building

In [ ]:
#past_sales_data_dummies.columns.drop(target_col)

In [ ]:
target_col_name = 'Sales'
input_cols_names = past_sales_data_dummies.columns.drop(target_col_name)

from sklearn.model_selection import train_test_split
train_x, validate_x, train_y, validate_y = train_test_split(past_sales_data_dummies[input_cols_names],
                                                   past_sales_data_dummies[target_col_name],
                                                   test_size=0.2,
                                                   random_state=1)
train_x.shape, validate_x.shape, train_y.shape, validate_y.shape

In [ ]:
#train_y.mean()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

base_model = DecisionTreeRegressor()
params = {'max_depth': list(range(5, 10))}
cv = GridSearchCV(estimator=base_model, param_grid=params).fit(train_x, train_y)

In [ ]:
#cv.best_params_

In [ ]:
from sklearn.metrics import mean_squared_error
validate_y_pred = cv.best_estimator_.predict(validate_x)
rmse = mean_squared_error(validate_y, validate_y_pred, squared=False)
print('RMSE on validation set is %.2f' % rmse)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression().fit(train_x, train_y)
validate_y_pred = lr_model.predict(validate_x)
rmse = mean_squared_error(validate_y, validate_y_pred, squared=False)
print('RMSE on validation set is %.2f' % rmse)

In [ ]:
'''
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100).fit(train_x, train_y)
validate_y_pred = rf_model.predict(validate_x)
rmse = mean_squared_error(validate_y, validate_y_pred, squared=False)
print('RMSE on validation set is %.2f' % rmse)
'''

In [ ]:
#!pip install pydotplus

In [ ]:
def draw_tree(model, columns):

    import pydotplus
    from six import StringIO
    from IPython.display import Image
    import os
    from sklearn import tree
    graphviz_path = 'C:\Program Files (x86)\Graphviz2.38/bin/'
    os.environ["PATH"] += os.pathsep + graphviz_path
    dot_data = StringIO()
    tree.export_graphviz(model,
                         out_file=dot_data,
                         feature_names=columns)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    return Image(graph.create_png())

In [ ]:
#draw_tree(cv.best_estimator_, train_x.columns)

## Submission

In [ ]:
#future_sales_data.head()

In [ ]:
future_sales_data_comb = pd.merge(left=future_sales_data, right=store_new,
                               how='left', on=['Store'])
future_sales_data_comb['StateHoliday'] = future_sales_data_comb['StateHoliday'].replace({0: '0'})
future_sales_data_comb['sales_month'] = future_sales_data_comb['Date'].dt.month
cols_ignore = ['Date', 'Store', 'Id']
future_sales_data_comb = future_sales_data_comb.drop(cols_ignore, axis=1)
future_sales_data_dummies = pd.get_dummies(future_sales_data_comb)
future_sales_data_dummies.shape, future_sales_data_comb.shape

In [ ]:
#past_sales_data['StateHoliday'].unique()

In [ ]:
#future_sales_data_comb['StateHoliday'].unique()

In [ ]:
#future_sales_data_comb.isna().sum()

In [ ]:
future_sales_data_dummies['StateHoliday_b'] = 0
future_sales_data_dummies['StateHoliday_c'] = 0
future_sales_data_dummies['Open'] = future_sales_data_dummies['Open'].fillna(1)
test_x = future_sales_data_dummies[input_cols_names]
test_y_pred = cv.best_estimator_.predict(test_x)
test_y_pred

In [ ]:
future_sales_data['Sales'] = test_y_pred
future_sales_data.head()

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/rossmann-store-sales/sample_submission.csv')
sample_submission.head()

In [ ]:
future_sales_data[['Id', 'Sales']].to_csv('submission.csv', index=False)